In [5]:
import pandas as pd
import sys
from sklearn.pipeline import Pipeline
import numpy as np
import string
import os
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import xgboost


ImportError: cannot import name 'XGBRegressor' from 'xgboost' (unknown location)

In [ ]:
os.path.abspath('..')
sys.path.append(os.path.abspath('..'))


In [ ]:
import importlib


import board_helpers
from board_helpers.move_info import MoveInfo
from board_helpers.tiles_counter import TilesCoutner
from board_helpers.board_consts import TileType
importlib.reload(board_helpers)
importlib.reload(board_helpers.move_info)
importlib.reload(board_helpers.tiles_counter)
importlib.reload(board_helpers.board_consts)



import transformers
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_rows_transformer import SelectRowsTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer
from transformers.map_set_transformer import MapSetTransformer
from transformers.one_hot_encoder_transformer import OneHotEncoderTransformer
from transformers.add_to_dict_transformer import AddToDictTransformer
from transformers.get_from_dict_transformer import GetFromDictTransformer
from transformers.select_pos_rows_transformer import SelectPosRowsTransformer
importlib.reload(transformers)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_rows_transformer)
importlib.reload(transformers.series_from_group_transformer)
importlib.reload(transformers.map_set_transformer)
importlib.reload(transformers.one_hot_encoder_transformer)
importlib.reload(transformers.add_to_dict_transformer)
importlib.reload(transformers.get_from_dict_transformer)
importlib.reload(transformers.select_pos_rows_transformer)

import functions
from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions)
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)

import processors
from processors.basic_pre_processor import BasicPreProcessor
importlib.reload(processors)
importlib.reload(processors.basic_pre_processor)

import builders
from builders.preprocessor_builder import PreprocessorBuilder
from builders.reg_pipe_builder import RegPipeBuilder
importlib.reload(builders)
importlib.reload(builders.preprocessor_builder)
importlib.reload(builders.reg_pipe_builder)

import searcher
import searcher_results_orgenizer
from searcher import Searcher
from searcher_results_orgenizer import SearcherResultsOrgenizer
importlib.reload(searcher)
importlib.reload(searcher_results_orgenizer)







<module 'searcher_results_orgenizer' from 'c:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble\\searcher_results_orgenizer.py'>

In [ ]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
turns = pd.read_csv('../data/features_filtered.csv', index_col='game_id')

In [ ]:
G_NAME = 'games'
T_NAME = 'turns'
DATA_NAME = 'train'

names = ['BetterBot', 'STEEBot', 'HastyBot']

prePipe = Pipeline([
                     ('get_relavent_games',  SelectRowsTransformer(train.index.unique(), target=G_NAME)),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=DATA_NAME, dest=DATA_NAME)),
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src=DATA_NAME, dest=G_NAME)),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src=DATA_NAME, dest=G_NAME)),
                    ('data_drop_bot_rating', SelectPosRowsTransformer('is_player', target=DATA_NAME)),
                    ])

In [ ]:
preprocessor = PreprocessorBuilder(games, G_NAME, turns, T_NAME, prePipe).build()
n_games, n_turns, n_ratings = preprocessor.process(train, DATA_NAME)

In [ ]:
n_data = n_games.merge(n_turns, left_index=True, right_index=True)

In [ ]:
featureTransformers = [('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                       ('hot', OneHotEncoderTransformer({'time_control_name': games['time_control_name'].unique(),
                                                          'game_end_reason':  games['game_end_reason'].unique(),
                                                          'lexicon':  games['lexicon'].unique(),
                                                          'rating_mode':  games['rating_mode'].unique(),
                                                          'bot_name': names}
                                                        )),
                       ('drops', NameDropperTransformer(['first', 'created_at', 'time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name'])),                                              
                      ]

In [ ]:
reg_pipe = Pipeline(featureTransformers)
t_data = reg_pipe.transform(n_data)

In [ ]:
grid = {
    'xgboost':
    {
    'estimator': XGBClassifier(random_state=0),
    'param_grid':
        {
        'max_depth': [None],
        },
    'scores': ['neg_root_mean_squared_error']
    },
    # 'random_forest_2':
    # {
    # 'estimator': RandomForestRegressor(random_state=0),
    # 'param_grid':
    #     {
    #     'max_depth': [2, 3],
    #     'max_features': ['sqrt', None]
    #     },
    # 'scores': ['neg_root_mean_squared_error']
    # },
}

In [ ]:
my_searcher = Searcher(reg_pipe, grid, 5, redo_scores=False)
my_searcher.search(n_data, n_ratings)

In [ ]:
res = SearcherResultsOrgenizer(redo_scores=False).orgenize(my_searcher.get_results())

In [ ]:
res.sort_values('mean_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,max_depth,max_features,min_samples_leaf
model_name,,,,,,,,,,,,,,,
random_forest_1,18.478862,0.200928,1.526418,0.301845,-164.364046,-164.225631,-163.142431,-163.875222,-163.555244,-163.832515,0.445418,25,40,log2,50
random_forest_1,22.701131,0.276059,1.438614,0.236246,-164.364046,-164.225631,-163.142431,-163.875222,-163.555244,-163.832515,0.445418,25,None,log2,50
random_forest_1,15.112928,0.196031,1.161658,0.213096,-164.364046,-164.225631,-163.142431,-163.875222,-163.555244,-163.832515,0.445418,25,80,log2,50
random_forest_1,20.347398,0.245278,1.351044,0.223468,-157.102981,-157.265023,-156.367003,-156.859820,-156.777381,-156.874442,0.307218,22,80,log2,20
random_forest_1,25.898070,0.695485,1.362553,0.171773,-157.102981,-157.265023,-156.367003,-156.859820,-156.777381,-156.874442,0.307218,22,None,log2,20
random_forest_1,21.967245,0.309466,1.577510,0.264294,-157.102981,-157.265023,-156.367003,-156.859820,-156.777381,-156.874442,0.307218,22,40,log2,20
random_forest_1,26.821495,1.186970,1.284311,0.224994,-149.471043,-149.730675,-148.468839,-148.939559,-148.941204,-149.110264,0.443671,20,None,log2,5
random_forest_1,26.624209,0.318622,1.596606,0.325981,-149.471043,-149.730675,-148.468839,-148.939559,-148.941204,-149.110264,0.443671,20,80,log2,5
random_forest_1,24.735457,0.487451,1.717256,0.308462,-149.465199,-149.749377,-148.408942,-149.081318,-148.801558,-149.101279,0.473715,19,40,log2,5
